In [58]:
train.shape

(5028, 905)

In [186]:
# coding: utf-8
# pylint: disable = invalid-name, C0111
import json
import lightgbm as lgb
import pandas as pd
import numpy
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score

# load or create your dataset
print('Load data...')
fulldataset = pd.read_csv('train.csv',sep=';')
columns = ['cash_in_out','display_type','scanner_code_reader']
fulldataset.drop(columns, inplace=True, axis=1)

test2 = pd.read_csv('test.csv', sep=';')
test2.drop(columns, inplace=True, axis=1)
test2 = test2.values

train, test = train_test_split(fulldataset, test_size=0.1, random_state=753)

target = train['target'].values
target_test = test['target'].values

#df_train2 = df_train[['cardreader_error_by_day','status_count']]
#columns = ['cash_in_out','display_type','scanner_code_reader']
#train.drop(columns, inplace=True, axis=1)
#train = train.values
train = train[['cardreader_error_by_day','status_count']].values
#train = lgb.Dataset('train.svm.txt')
#train.save_binary('train.bin')

#df_test2 = df_test[['cardreader_error_by_day','status_count']]
#test.drop(columns, inplace=True, axis=1)
#test = test.values
test = test[['cardreader_error_by_day','status_count']].values
# create dataset for lightgbm
lgb_train = lgb.Dataset(train, target)
lgb_eval = lgb.Dataset(test, target_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 5,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

#gbm = lgb.train(params, lgb_train, num_boost_round=20)
print('Save model...')
# save model to file
gbm.save_model('model.txt')
#print('Start predicting...')
# predict
PREDICT = gbm.predict(test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(target_test, PREDICT) ** 0.5)
print('F-score:',f1_score(y_pred=numpy.around(PREDICT),y_true=target_test))

print('Finish')
PREDICT[0:10]

Load data...
Start training...
[1]	valid_0's auc: 0.581605	valid_0's l2: 0.249359
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.579764	valid_0's l2: 0.249212
[3]	valid_0's auc: 0.581124	valid_0's l2: 0.248641
[4]	valid_0's auc: 0.579764	valid_0's l2: 0.248517
[5]	valid_0's auc: 0.590441	valid_0's l2: 0.247917
[6]	valid_0's auc: 0.589899	valid_0's l2: 0.247794
[7]	valid_0's auc: 0.590011	valid_0's l2: 0.247352
[8]	valid_0's auc: 0.590031	valid_0's l2: 0.247248
[9]	valid_0's auc: 0.590011	valid_0's l2: 0.246853
[10]	valid_0's auc: 0.590031	valid_0's l2: 0.246766
Early stopping, best iteration is:
[5]	valid_0's auc: 0.590441	valid_0's l2: 0.247917
Save model...
The rmse of prediction is: 0.49791282746
F-score: 0.477551020408
Finish


array([ 0.50365752,  0.49806971,  0.50365752,  0.50365752,  0.49806971,
        0.50999842,  0.50504342,  0.49806971,  0.48679119,  0.48679119])

In [187]:
PREDICT = gbm.predict(test2, num_iteration=gbm.best_iteration)
OUT=pd.DataFrame(PREDICT, columns=['PREDICT']) 
OUT.to_csv('OUT.csv', sep=';', encoding='utf-8')
PREDICT[0:10]

array([ 0.47192945,  0.47459209,  0.47459209,  0.47459209,  0.47459209,
        0.47459209,  0.47459209,  0.47459209,  0.47459209,  0.47459209])

In [189]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
final_col_names = fulldataset.columns
fulldataset = fulldataset.values
fulldataset[:, cont_feats_idx] = scaler.fit_transform(fulldataset[:, cont_feats_idx])

scaler_2 = MinMaxScaler(feature_range=(0, 1))
fulldataset[:, int_feats_idx] = scaler_2.fit_transform(fulldataset[:, int_feats_idx])

NameError: name 'cont_feats_idx' is not defined

In [191]:
# Extract target before scaling
labels = fulldataset.pop('TARGET')
labels = labels[:len_train]

# Reshape (one-hot)
target = np.zeros([len(labels), len(np.unique(labels))])
target[:, 0] = labels == 0
target[:, 1] = labels == 1

TypeError: 'numpy.ndarray' object is not callable